In [15]:
import os
import shutil
from tqdm import tqdm
from model_training.measures import summary
from backend.intercom_classifier import WordAudioClassifier

## Convert Collected Data to Required File Structure

In [2]:
def split_files_by_category(input_path, output_path):
    """
    Splits files into subfolders based on the {word} part of their filenames.
    
    Parameters:
        input_path (str): Path to the input directory containing files to split.
        output_path (str): Path to the output directory where subfolders will be created.
    """
    if not os.path.exists(input_path):
        print(f"Error: Input path '{input_path}' does not exist.")
        return

    os.makedirs(output_path, exist_ok=True)

    for filename in tqdm(os.listdir(input_path)):
        file_path = os.path.join(input_path, filename)

        if not os.path.isfile(file_path):
            continue

        try:
            base_name, _ = os.path.splitext(filename)
            word, _ = base_name.split("_")
        except ValueError:
            print(f"Skipping file '{filename}' (does not match the required pattern).")
            continue

        category_folder = os.path.join(output_path, word)
        os.makedirs(category_folder, exist_ok=True)

        destination_path = os.path.join(category_folder, filename)
        shutil.copy(file_path, destination_path)

In [3]:
split_files_by_category('../data/manual', '../data/manual_split')

100%|██████████| 60/60 [00:00<00:00, 104.40it/s]


## Evaluate the Model on Collected Data

In [8]:
CLASS_LABELS = os.listdir('../data/manual_split')

classifier = WordAudioClassifier(
    class_labels=CLASS_LABELS,
    model_path="../models/robustness_3_single_cnn_tsrc_spectrograms_cleaned_augmented_sharpened/",
)

Model loaded successfully


In [9]:
y_true = []
y_pred = []

for word in tqdm(CLASS_LABELS):
    word_folder = os.path.join('../data/manual_split', word)
    for file in os.listdir(word_folder):
        file_path = os.path.join(word_folder, file)
        predicted_label = classifier.predict_word(file_path)

        y_true.append(CLASS_LABELS.index(word))
        y_pred.append(CLASS_LABELS.index(predicted_label))

  0%|          | 0/30 [00:00<?, ?it/s]E:\MINI8\ITM\intro-to-ml\backend\audio_cleaner.py:37: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(input_path, sr=None)
e:\MINI8\ITM\intro-to-ml\venv\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
100%|██████████| 30/30 [00:17<00:00,  1.75it/s]


In [16]:
summary(y_true, y_pred, CLASS_LABELS)

Accuracy: 0.7833333333333333
Micro F1 Score: 0.7833333333333333
Macro F1 Score: 0.7944444444444444
Weighted F1 Score: 0.7944444444444445


E:\MINI8\ITM\intro-to-ml\model_training\measures.py:138: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
